## CustomData to try

In [17]:
custom_dict = {'UserID':[1, 1, 1, 1, 1, 2, 2, 3],
               'OrderID':[100, 100, 100, 200, 200, 300, 400, 500],
               'ItemID':[10, 11, 11, 12, 15, 15, 16, 17],
               'Description':[' McCombo 12 with Double Cheeseburger with bacons',
                             'McToast, hashbrow_ 4 and_ a drink_ of choice ',
                             'Happy Meal-with Pancakes and a drink of choice',
                             'Double File-o-Fishs',
                             'Cheeseburgers_',
                             'Cheeseburgers_',
                             'McCombo with_ Caesar_ Roll',
                             'Happy Meal-with Pancakes and a drink of choice'],
              'Count' :[1, 1, 1, 3, 4, 1, 2, 1]}
custom_data = pd.DataFrame(custom_dict)

In [1]:
path = "../../data/processed/userItems.csv"

## Loading and previewing data

In [6]:
import pandas as pd
import numpy as np
import spacy
import re

custom_data = pd.read_csv(path)

In [7]:
# python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

def normalize(text):
    text = re.sub(r'[^a-zA-Z ]', '', text) # remove all non latin characters 
    text = nlp(text.lower())
    tokens = [token.lemma_ for token in text if len(token) > 2 and  # leave raw tokens if length > 2
                                                not token.is_stop and # and if token is not a stop word
                                                token.pos_ != 'ADJECTIVE'] # and it is a NOUN
    tokens = list(set(tokens))  # delete repeating words
    return ' '.join(tokens) #returning clear text description
    
def tokenize(text):
    return text.split()

In [9]:
custom_data['Description'] = custom_data['Description'].astype(str)

In [ ]:

custom_data['Description'] = custom_data['Description'].apply(normalize)  # all data in 'description' must be str, otherwise exception!
custom_data

### Save clear description data

In [ ]:
custom_data.to_csv('../../data/processed/userItemsClearDesc.csv')

## Vectorizing

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
vectorizer = CountVectorizer()
categories_data = vectorizer.fit_transform(custom_data['Description'])
categories_voc = vectorizer.vocabulary_

In [28]:
custom_data['categories'] = [np.array(row) for row in categories_data.toarray()]
custom_data['categories'] = custom_data['categories'] * custom_data['Count']
custom_data.drop(['Description', 'Count', 'ItemID'], axis=1, inplace=True)

In [29]:
custom_data.head(5)

,UserID,OrderID,categories
0,1,100,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
1,1,100,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0]"
2,1,100,"[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]"
3,1,200,"[0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0]"
4,1,200,"[0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


## GroupBy UserID, OrderID, ItemID

In [30]:
cat = sorted(categories_voc.items(), key=lambda kv:kv[1])
cat = [c[0] for c in cat]
custom_data[cat] = pd.DataFrame(custom_data['categories'].values.tolist(), index = custom_data.index)
custom_data.drop('categories', axis=1, inplace=True)

In [31]:
categorized_custom_data =custom_data.groupby(['UserID', 'OrderID']).sum().groupby('UserID').mean()

In [32]:
categorized_custom_data.head(5)

,bacon,caesar,cheeseburger,choice,double,drink,fileofish,happy,hashbrow,mccombo,mctoast,mealwith,pancake,roll
UserID,,,,,,,,,,,,,,
1,0.5,0.0,2.5,1.0,2.0,1.0,1.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0
2,0.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
